# Secretary Module - Interactive Testing Notebook

Test the Secretary module with your own inputs!

## Setup

In [ ]:
# Install dependencies (run once)
!pip install -q ipywidgets

In [ ]:
# Import the Secretary module
from secretary import Secretary, create_full_workflow
import json

print("✅ Secretary module loaded!")

## Test 1: Simple Validation

**Edit the requirements below and run to test validation:**

In [ ]:
# ===== EDIT YOUR REQUIREMENTS HERE =====
test_requirements = {
    'topic': 'Introduction to Python',
    'style': 'tutorial',  # Options: funny, documentary, serious, graphic-heavy, tutorial
    'duration_range': '5 minutes',
    'audio_mode': 'generate',  # generate or upload
    'script_mode': 'generate',  # generate or upload
    'comments': 'Make it beginner-friendly'
}
# ========================================

# Test validation
sec = Secretary()
is_valid, errors = sec.validate_inputs(test_requirements)

print("\nRequirements:")
print(json.dumps(test_requirements, indent=2))
print("\n" + "="*50)

if is_valid:
    print("✅ VALIDATION PASSED!")
else:
    print("❌ VALIDATION FAILED!")
    print("\nErrors:")
    for error in errors:
        print(f"  - {error}")

## Test 2: Multiple Test Cases

Run this to test several scenarios at once:

In [ ]:
test_cases = [
    {
        "name": "Valid - Tutorial",
        "data": {
            'topic': 'Machine Learning Basics',
            'style': 'tutorial',
            'duration_range': '5-7 minutes',
            'audio_mode': 'generate',
            'script_mode': 'generate',
            'comments': 'Focus on beginners'
        }
    },
    {
        "name": "Valid - Custom Style",
        "data": {
            'topic': 'Climate Change',
            'style': 'Educational and engaging',
            'duration_range': '3-5 minutes',
            'audio_mode': 'upload',
            'script_mode': 'generate',
            'comments': None
        }
    },
    {
        "name": "Invalid - Topic Too Short",
        "data": {
            'topic': 'AI',  # Only 2 chars!
            'style': 'tutorial',
            'duration_range': '5 minutes',
            'audio_mode': 'generate',
            'script_mode': 'generate',
            'comments': None
        }
    },
    {
        "name": "Invalid - Bad Duration",
        "data": {
            'topic': 'Valid Topic',
            'style': 'funny',
            'duration_range': 'five minutes',  # Wrong format!
            'audio_mode': 'generate',
            'script_mode': 'upload',
            'comments': None
        }
    },
]

sec = Secretary()

for test_case in test_cases:
    print("\n" + "="*60)
    print(f"Test: {test_case['name']}")
    print("="*60)
    
    is_valid, errors = sec.validate_inputs(test_case['data'])
    
    if is_valid:
        print("✅ VALID")
    else:
        print("❌ INVALID")
        for error in errors:
            print(f"  - {error}")

## Test 3: Generate a Script

Test the scriptwriter tool:

In [ ]:
# ===== EDIT YOUR REQUIREMENTS HERE =====
my_requirements = {
    'topic': 'Python Programming Tips',
    'style': 'tutorial',
    'duration_range': '5 minutes',
    'audio_mode': 'generate',
    'script_mode': 'generate',
    'comments': 'Include code examples'
}
# ========================================

sec = Secretary()

# Validate first
is_valid, errors = sec.validate_inputs(my_requirements)

if not is_valid:
    print("❌ Invalid requirements:")
    for error in errors:
        print(f"  - {error}")
else:
    print("✅ Requirements validated!\n")
    
    # Call scriptwriter
    print("Calling ScriptWriter...\n")
    result = sec.call_tool('scriptwriter', my_requirements)
    
    if result['success']:
        print("✅ Script generated!\n")
        print("="*60)
        print("GENERATED SCRIPT:")
        print("="*60)
        print(result['outputs']['script'])
        print("\n" + "="*60)
        print(f"Word count: {result['outputs']['word_count']}")
    else:
        print(f"❌ Failed: {result['error']}")

## Test 4: Call Multiple Tools

Test scriptwriter → audio agent → langsearch:

In [ ]:
# ===== EDIT YOUR REQUIREMENTS HERE =====
requirements = {
    'topic': 'Artificial Intelligence Ethics',
    'style': 'documentary',
    'duration_range': '3-5 minutes',
    'audio_mode': 'generate',
    'script_mode': 'generate',
    'comments': 'Include real-world examples'
}
# ========================================

sec = Secretary()

# Step 1: Generate script
print("Step 1: Generating script...")
script_result = sec.call_tool('scriptwriter', requirements)

if script_result['success']:
    print("✅ Script generated!\n")
    script = script_result['outputs']['script']
    
    # Step 2: Generate audio
    print("Step 2: Generating audio...")
    audio_result = sec.call_tool('audio_agent', {'script': script})
    
    if audio_result['success']:
        print("✅ Audio generated!")
        print(f"   Audio file: {audio_result['outputs']['audio_file']}")
        print(f"   Duration: {audio_result['outputs']['duration']}s\n")
        
        # Step 3: Research terms
        print("Step 3: Researching terms...")
        research_result = sec.call_tool('langsearch', {'script': script})
        
        if research_result['success']:
            print("✅ Research complete!")
            research_data = research_result['outputs']['research_data']
            print(f"   Terms found: {research_data['total_terms_found']}\n")
            
            print("="*60)
            print("✅ ALL STEPS COMPLETED SUCCESSFULLY!")
            print("="*60)
        else:
            print(f"❌ Research failed: {research_result['error']}")
    else:
        print(f"❌ Audio failed: {audio_result['error']}")
else:
    print(f"❌ Script failed: {script_result['error']}")

## Test 5: Full Workflow (All 6 Workers)

Run the complete pipeline from requirements to final video:

In [ ]:
# ===== EDIT YOUR REQUIREMENTS HERE =====
full_requirements = {
    'topic': 'Space Exploration Milestones',
    'style': 'documentary',
    'duration_range': '5 minutes',
    'audio_mode': 'generate',
    'script_mode': 'generate',
    'comments': 'Include NASA missions and future plans'
}
# ========================================

sec = Secretary()

# Set and validate
is_valid, errors = sec.set_requirements(full_requirements)

if not is_valid:
    print("❌ Invalid requirements:")
    for error in errors:
        print(f"  - {error}")
else:
    print("✅ Requirements validated!\n")
    
    # Create full workflow
    workflow = create_full_workflow(sec.requirements)
    
    print(f"Workflow has {len(workflow)} steps:")
    for idx, (tool_name, _) in enumerate(workflow):
        print(f"  {idx + 1}. {tool_name}")
    
    print("\nExecuting workflow...\n")
    print("="*60)
    
    # Execute
    result = sec.execute_workflow(workflow)
    
    if result['success']:
        print("\n" + "="*60)
        print("✅ WORKFLOW COMPLETED SUCCESSFULLY!")
        print("="*60)
        print(f"\nCompleted: {result['completed_steps']}/{result['total_steps']} steps\n")
        
        # Show each step's outputs
        print("Step Results:")
        print("-"*60)
        for step_idx, step_data in result['results'].items():
            tool_name = step_data['tool']
            outputs = list(step_data['result']['outputs'].keys())
            print(f"  Step {step_idx + 1}: {tool_name}")
            print(f"    Outputs: {', '.join(outputs)}")
        
        # Show final video info
        final = result['results'][5]['result']['outputs']
        print("\n" + "="*60)
        print("FINAL VIDEO:")
        print("="*60)
        print(f"  File: {final['video_file']}")
        print(f"  Duration: {final['duration']}s")
        print(f"  Resolution: {final['resolution']}")
        print(f"  Size: {final['file_size_mb']} MB")
    else:
        print(f"\n❌ Workflow failed at step {result['failed_step'] + 1}")
        failed = result['results'][result['failed_step']]
        print(f"Failed tool: {failed['tool']}")
        print(f"Error: {failed['result']['error']}")

## Test 6: Your Custom Test

**Create your own test here - edit and experiment!**

In [ ]:
# Your custom test here!
# Try different topics, styles, durations, etc.

my_test = {
    'topic': 'Your topic here',
    'style': 'funny',
    'duration_range': '2-3 minutes',
    'audio_mode': 'generate',
    'script_mode': 'upload',
    'comments': 'Your comments'
}

sec = Secretary()
is_valid, errors = sec.validate_inputs(my_test)

print("Result:", "✅ VALID" if is_valid else "❌ INVALID")
if errors:
    for error in errors:
        print(f"  - {error}")

## Quick Reference

**Valid Styles:**
- `funny`
- `documentary`
- `serious`
- `graphic-heavy`
- `tutorial`
- Or any custom string (min 3 chars)

**Valid Duration Formats:**
- `"5 minutes"`
- `"2-3 minutes"`
- `"30 seconds"`
- `"30-60 seconds"`

**Audio/Script Modes:**
- `"generate"`
- `"upload"`

**Validation Rules:**
- Topic: min 5 characters
- Style: preset OR custom (min 3 chars)
- Duration: must match format above
- Audio/Script: exactly "generate" or "upload"
- Comments: optional (any text or None)